### Use Google API to read Google sheet into Pandas dataframe

**Saved here for use in case**

```
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from gspread_dataframe import get_as_dataframe
import pandas as pd
import numpy as np
 
#Use creds to create a client to interact with the Google Drive API
scope = ['https://spreadsheets.google.com/feeds']
creds = ServiceAccountCredentials.from_json_keyfile_name('client_secret.json', scope)
client = gspread.authorize(creds)
 
#Find a workbook by name and open the first sheet
#Make sure you use the right name here.
sheet = client.open("pubmedarticle").sheet1
 
#convert into dataframe
pub_article = get_as_dataframe(sheet)

print(df.head()) ```

### Read in the major PubMed article file: pubmedarticle.csv

In [1]:
import pandas as pd
import numpy as np

directory = "E:/Pubmed_mindfulness/"
filename = directory + "pubmedarticle.csv"

pub_article = pd.read_csv(filename, na_values=['NULL'])
print(pub_article.shape)
print("The total number of rows:",len(pub_article))
pub_article.head()

(4595, 46)
Total row number: 4595


,PubmedArticleID,PMID,DataSetID,DateCreated,DateCreatedYear,PubDate,PubDateYear,CitationOwner,CitationStatus,ArticleTitle,...,aCity,PubModel,ArticleDateType,ArticleDateYear,PubYearPrint,PubYearElectronic,PubPreferredCiteYear,clinTrialRegistry,funderList,funderBreakdown
0,198887,28963884,164,2017-09,2017,2017-Aug,2017,NLM,Publisher,"Yoga, mindfulness-based stress reduction and s...",...,NaN,Print-Electronic,Electronic,2017.0,2017,2017.0,2017,NaN,NaN,No funders are listed
1,198888,28962535,164,2017-09,2017,2017-Sep,2017,NLM,Publisher,Using Art For Health Promotion: Evaluating an ...,...,NaN,Print-Electronic,Electronic,2017.0,2017,2017.0,2017,NaN,NaN,No funders are listed
2,198889,28961631,164,2017-09,2017,2017-Sep,2017,NLM,Publisher,Does mindfulness meditation improve chronic pa...,...,NaN,Print-Electronic,Electronic,2017.0,2017,2017.0,2017,NaN,NaN,No funders are listed
3,198890,28959369,164,2017-09,2017,2017-Oct,2017,NLM,PubMed-not-MEDLINE,Effects of Tai Chi versus Physical Therapy on ...,...,NaN,Print-Electronic,Electronic,2017.0,2017,2017.0,2017,NaN,NaN,No funders are listed
4,198891,28955285,164,2017-09,2017,NaN,2017,NLM,PubMed-not-MEDLINE,The Buffering Effect of Mindfulness on Abusive...,...,NaN,Electronic-eCollection,Electronic,2017.0,NaN,2017.0,2017,NaN,NaN,No funders are listed


In [2]:
# Take the column names of a dataframe and make the names be a list for further processing
var_list = list(pub_article)
print("List all the columns:",var_list)

List all the columns: ['PubmedArticleID', 'PMID', 'DataSetID', 'DateCreated', 'DateCreatedYear', 'PubDate', 'PubDateYear', 'CitationOwner', 'CitationStatus', 'ArticleTitle', 'JournalTitle', 'Volume', 'Issue', 'Pagination', 'Abstract', 'CitedMedium', 'Affiliation', 'Language', 'JournalNlmUniqueID', 'UserRecordNote', 'JournalCountry', 'Fauthor', 'Fchemical', 'Fgrantinfo', 'Fmesh', 'Fpubtype', 'Farticledata', 'Fhistory', 'FTpmc', 'aSchoolColl', 'aDept', 'aCountry', 'aState', 'Fpopulations', 'FmeshMajor', 'aInstitution', 'aCity', 'PubModel', 'ArticleDateType', 'ArticleDateYear', 'PubYearPrint', 'PubYearElectronic', 'PubPreferredCiteYear', 'clinTrialRegistry', 'funderList', 'funderBreakdown']


### Describe the datatypes of the columns to see whether any needs to be set as string

**Create a variable list which presents the dataframe columns which: **
1. carries values
2. are supposed to be string

In [9]:
# Use select_dtypes() method to select the numeric columns. The output is in the type of a dataframe.
pub_numeric = pub_article.select_dtypes(include = [np.number])

# Select the row with the index label "count" from the description dataframe.The output is a Pandas series.
# This row now as a series has index labels which represent the original dataframe variables/columns.
subSeries = pub_numeric.describe().loc["count"]

# Because the columns of the numeric dataframe are supposed to be numeric, 
# that count == 0.0 means all the values are missing.These columns should be dropped.
varlist_miss = subSeries[subSeries == 0.0].index.tolist() # variable list used to drop the columns.

print("All missing values:",varlist_miss)

# The result of this step is a list, varlist_nomiss of which the elements represent the dataframe columns
# which has at least one non-missing value.
# This list is needed for converting the values of the columns in the original dataframe into string.
varlist_nomiss = subSeries[subSeries > 0].index.tolist() # variable list used to convert the numeric to be string

print("No missing values:",varlist_nomiss)

All missing values: ['Pagination', 'Abstract', 'Affiliation', 'UserRecordNote', 'Fauthor', 'Fchemical', 'Fgrantinfo', 'Fmesh', 'Fpubtype', 'Farticledata', 'Fhistory', 'aSchoolColl', 'aDept', 'aCountry', 'aState', 'Fpopulations', 'FmeshMajor', 'aInstitution', 'aCity', 'clinTrialRegistry']
No missing values: ['PubmedArticleID', 'PMID', 'DataSetID', 'DateCreatedYear', 'ArticleDateYear', 'PubYearElectronic']


E:\WinPython-64bit-3.4.4.4Qt5\python-3.4.4.amd64\lib\site-packages\numpy\lib\function_base.py:4269: RuntimeWarning: Invalid value encountered in percentile
  interpolation=interpolation)


In [4]:
# Drop the columns with all missing values.
temp = pub_article.drop(varlist_miss, axis=1)

print("List all the columns each of which carries at least one value:",list(temp)) # columns with values.
print()
print("The total number of columns in dataframe:",len(list(temp)))

List all the columns each of which carries at least one value: ['PubmedArticleID', 'PMID', 'DataSetID', 'DateCreated', 'DateCreatedYear', 'PubDate', 'PubDateYear', 'CitationOwner', 'CitationStatus', 'ArticleTitle', 'JournalTitle', 'Volume', 'Issue', 'CitedMedium', 'Language', 'JournalNlmUniqueID', 'JournalCountry', 'FTpmc', 'PubModel', 'ArticleDateType', 'ArticleDateYear', 'PubYearPrint', 'PubYearElectronic', 'PubPreferredCiteYear', 'funderList', 'funderBreakdown']

The total number of columns in dataframe: 26


**2. Construct a new dataframe named "temp" by converting the ID and date-related numeric columns into string**

In [5]:
for var in varlist_nomiss:
    if (var.find("ID") > -1) or (var.find("Date") > -1) or (var.find("Year") > -1) or \
    (var.find("Month") > -1) or (var.find("Day") > -1):
        temp[var] = temp[var].astype(str)
        print(temp[var].describe())
        print()

count       4595
unique      4595
top       200597
freq           1
Name: PubmedArticleID, dtype: object

count         4595
unique        4595
top       22973242
freq             1
Name: PMID, dtype: object

count     4595
unique       1
top        164
freq      4595
Name: DataSetID, dtype: object

count     4595
unique      33
top       2016
freq       975
Name: DateCreatedYear, dtype: object

count     4595
unique      15
top        nan
freq      1657
Name: ArticleDateYear, dtype: object

count     4595
unique      15
top        nan
freq      1657
Name: PubYearElectronic, dtype: object



** Write a function to summarize & describe a string variable/column and make the output a dataframe.**

In [6]:
def str_col_describe(column,df):
    # Calcuate count & make the output as a dataframe.
    col_freq = pd.DataFrame(df[column].value_counts().reset_index())
    
    # Rename the columns of the output dataframe.
    col_freq.columns = [column, "Count"]
    
    # Create a new column named "%" by calculating the rounded percentage multipled by 100.
    total = sum(col_freq["Count"])
    col_freq["%"] = round(100*col_freq["Count"]/total, 2)
    
    return col_freq, total

In [7]:
# Describe all the columns after deleting the empty ones.         
print("All the columns with values-\n",temp.describe(include = [np.object]))

# Selectively look into some columns.
col_exam = ['DateCreated', 'DateCreatedYear', 'PubDate', 'PubDateYear', 'CitationOwner', 'CitationStatus', 'CitedMedium', 'Language', 'JournalCountry', 'FTpmc', 'PubModel', 'ArticleDateType', 'ArticleDateYear', 'PubYearPrint', 'PubYearElectronic', 'PubPreferredCiteYear', 'funderList', 'funderBreakdown']

for e in col_exam:
    a, b = str_col_describe(e,temp)
    print(a)
    print("Total of count=", b)
    print()

All the columns with values-
        PubmedArticleID      PMID DataSetID DateCreated DateCreatedYear  \
count             4595      4595      4595        4595            4595   
unique            4595      4595         1         226              33   
top             200597  22973242       164     2016-09            2016   
freq                 1         1      4595         111             975   

         PubDate PubDateYear CitationOwner CitationStatus  \
count       3858        4595          4595           4595   
unique       275          36             2              5   
top     2017-Aug        2016           NLM        MEDLINE   
freq          90         838          4591           3151   

                                             ArticleTitle  \
count                                                4593   
unique                                               4571   
top     Attending to weak signals: the leader's challe...   
freq                                             

In [32]:
# Clean a value in the column PubDateYear, change "Spring 2017" to be "2017".
temp.loc[temp["PubDateYear"] == "Spring 2017", "PubDateYear"] = "2017"

c, d = str_col_describe('PubDateYear',temp)
print(c)
print("Total of count:",d)

   PubDateYear  Count      %
0         2016    838  18.24
1         2017    768  16.71
2         2015    716  15.58
3         2014    545  11.86
4         2013    429   9.34
5         2012    306   6.66
6         2011    248   5.40
7         2010    191   4.16
8         2009    154   3.35
9         2008    102   2.22
10        2007     86   1.87
11        2006     47   1.02
12        2004     36   0.78
13        2005     31   0.67
14        2003     23   0.50
15        2002     14   0.30
16        2001     13   0.28
17        1999      9   0.20
18        2000      7   0.15
19        1998      5   0.11
20        1995      4   0.09
21        1997      3   0.07
22        1985      3   0.07
23        1984      2   0.04
24        1992      2   0.04
25        1989      2   0.04
26        2018      2   0.04
27        1994      2   0.04
28        1988      1   0.02
29        1980      1   0.02
30        1987      1   0.02
31        1993      1   0.02
32        1996      1   0.02
33        1990

### Test a Pandas pivot table / Cross table

In [33]:
print("Frequency Table (Language x PubModel):")
pd.crosstab(temp.Language, temp.PubModel, margins=True)

Frequency Table (Language x PubModel):


PubModel,Electronic,Electronic-Print,Electronic-eCollection,Print,Print-Electronic,All
Language,,,,,,
chi,0,0,0,2,0,2
dan,0,0,0,3,0,3
dut,0,0,0,5,0,5
eng,262,11,314,1554,2319,4460
fin,0,0,0,1,0,1
fre,0,0,0,33,9,42
ger,0,0,0,19,16,35
gre,0,0,0,1,0,1
heb,0,0,0,3,0,3


In [34]:
print("Row Percentage (Language x PubModel):")
pd.crosstab(temp.Language, temp.PubModel, normalize = "index", margins=True)

Row Percentage (Language x PubModel):


PubModel,Electronic,Electronic-Print,Electronic-eCollection,Print,Print-Electronic
Language,,,,,
chi,0.000000,0.000000,0.000000,1.000000,0.000000
dan,0.000000,0.000000,0.000000,1.000000,0.000000
dut,0.000000,0.000000,0.000000,1.000000,0.000000
eng,0.058744,0.002466,0.070404,0.348430,0.519955
fin,0.000000,0.000000,0.000000,1.000000,0.000000
fre,0.000000,0.000000,0.000000,0.785714,0.214286
ger,0.000000,0.000000,0.000000,0.542857,0.457143
gre,0.000000,0.000000,0.000000,1.000000,0.000000
heb,0.000000,0.000000,0.000000,1.000000,0.000000
